<img src="files/Pics/LOGOS.png" width="800">

# Labor 10 - Spam szűrés

Ebbean a labor gyakorlatban emailek spam szűrésére fogjuk alkalmazni az SVM algoritmust. 

### 1: Beolvasás

Olvassuk be az adatainkat. Két emaillel fogunk dolgozni, illetve rendelkezésünkre áll egy szótár, ami a leggyakrabban használt kifejezések normalizált alakját használja. 

In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.svm import SVC
import re
from nltk.stem import PorterStemmer             # natural langage toolkit

mail1 = open("emailSample1.txt","r").read()     # első mail betöltése
mail2 = open("emailSample2.txt","r").read()     # második mail betöltése
vocabList = open("vocab.txt","r").read()        # szótár betöltése

print('First mail:')                            # kiíratások
print(mail1)
print('Second mail:')
print(mail2)
print('Vocabulary list:')
print(vocabList)

vocabList=vocabList.split("\n")[:-1]            # kicsit átalakítjuk a könnyebb kezelésért
vocabList_d={}
for ea in vocabList:
    value,key = ea.split("\t")[:]
    vocabList_d[key] = value

### 2: Email előfeldolgzás

Első lépésként normalizálnunk kell az email szövegét. Mit is jelent ez?
- Mindent kisbetűssé konverálunk
- Kivesszük a HTML kódokat
- Normalizáljuk az URL címeket
- Normalizáljuk a számokat
- Normalizáljuk az email címeket
- Normalizáljuk a speciális charactereket
- A szavakat szótőre redukáljuk
- Névelők elhagyása (egybetűs karakterek)

A normalizálás a legtöbb esetben azt fogja jelenteni, hogy egy egyszerűsített stringel helyettesítjük az adott elemet.

Normalizálás után pedig egy számsorozattá dekódoljuk az emailt a szótárunk alapján. Vagyis vissza adjuk az emailbe szereplő szavak indexét a listánkból.

In [ ]:
def processEmail(mailcontent):
    word_indices=[]                                                         # inicializálás
    
    mailcontent = mailcontent.lower()                                       #kisbetűk
    mailcontent = re.sub("[http|https]://[^\s]*","httpaddr",mailcontent)    # HTML normalizálása
    mailcontent = re.sub("[^\s]+@[^\s]+","emailaddr",mailcontent)           # email-címek normalizálása 
    mailcontent = re.sub("[0-9]+","number",mailcontent)                     # számok normalizálása
    specChar = ["<","[","^",">","+","?","!","'",".",",",":","$"]            # speciális karakterek
    #####################################################################
    # speciális karakterek normalizálása
    
    
    #####################################################################
    
    ps = PorterStemmer()                                                    # természetesnyelv feldolgozás szótövek
    mailcontent = [ps.stem(token) for token in mailcontent.split(" ")]
    mailcontent= " ".join(mailcontent)
    
    mailcontent = mailcontent.replace("\n"," ")
    
    #####################################################################
    # word_indices feltöltése = Email dekódolása szám listává
    
    
    
    
    #####################################################################
    
    print('Preprocessed mail:',mailcontent)
    return word_indices

word_indices = processEmail(mail1)
print('\nWord indices:',word_indices)

check_email= [86, 916, 794, 1077, 883, 370, 1699, 790, 1822, 1831, 883, 431, 1171, 794, 1002, 1893, 1364, 592, 1676, 238, 162, 89, 688, 945, 1663, 1120, 1062, 1699, 375, 1162, 1120, 1893, 1510, 1182, 1237, 810, 1895, 1440, 1547, 181, 1699, 1758, 1896, 688, 1676, 992, 961, 1477, 71, 530, 1699, 531]
if word_indices == check_email:
    print("\nSikeres átalakítás. Tovább mehet.")
else:
    print("\nSikertelen átalakítás. Módosítás szükséges.")

### 3: Feature extraction

A következő lépésben az előfeldolgozott emailből hozzunk létre egy features vektort, aminek a mérete megegyezik a szótárunk méretével és amelyik szó a szótárból szerepel az emailben annak a feauturnek az értéke 1-es legyen.

<img src="files/Pics/L10_vector.png" width="150">

In [ ]:
def emailFeatures(word_indices,vocabList):
#####################################################################   
    
    
    
    
#####################################################################
    return features

features = emailFeatures(word_indices,vocabList_d)
print("Length of feature vector (1899 expected): %.0f" % len(features))
print("Number of non-zero entries (43 expected): %.0f" % np.sum(features))

### 4: SVM tanítása a Spam klasszifikálására

A training emailen tanítsuk be a SVM classificatort lineáris kernelt használva.

In [ ]:
spam_mat = loadmat("spamTrain.mat")
X_train = spam_mat["X"]
y_train = spam_mat["y"]
C =0.2

#####################################################################
#SVC látrehozása és illesztése (kernel=linear), ravel() függvényt ne felejtsük el használni.



#####################################################################

print("Training Accuracy (99.975% expected):",(spam_predictor.score(X_train,y_train.ravel()))*100,"%")

### 5: Klasszifikáció tesztelése

In [ ]:
spam_mat2 = loadmat("spamTest.mat")
X_test = spam_mat2["Xtest"]
y_test = spam_mat2["ytest"]

print("Training Accuracy (98.9% expected):",(spam_predictor.score(X_test,y_test.ravel()))*100,"%")

### 6: A Spam legfőbb jelzői
Mivelhogy a modell, amit tanítunk egy lineáris SVM, megnézhetjük, az egyes súlyokat, amit a modell megtanult a klasszifikálás során. A következőkben egy olyan kódrészletet implementálunk, ami megmutatja, melyek azok a szava (és súlyaik), melyekről az algoritmus leginkább "gondolja", hogy spam.

In [ ]:
weights = spam_predictor.coef_[0]
weights_col = np.hstack((np.arange(1,1900).reshape(1899,1),weights.reshape(1899,1)))
weights_sorted = weights_col[weights_col[:,1].argsort()][::-1]

spamvoc_ind = weights_sorted[0:15,0]
spamvoc_weights = (weights_sorted[0:15,1])
j=0
for i in spamvoc_ind:
    print(vocabList[int(i-1)],'\t', '\t', spamvoc_weights[j])
    j=+1



### 7: Saját mail tesztelése

A kíváncsiság kedvéért akér saját emailünkön is tesztelhetjük a dolgot.

In [ ]:
ownmail = open("spamSample1.txt","r").read()
own_ind = processEmail(ownmail)
x = emailFeatures(own_ind,vocabList_d)

p = spam_predictor.predict(x.reshape(1,-1))
print("Result is:",p)
if (p==0):
    print("This is NOT a SPAM")
elif(p==1):
    print("This is a SPAM")

